# Data Exploration - Election Commission of India Voter Turnout Data

## Assignment: Data Visualization and Story Telling

This notebook explores the PC-wise voter turnout data from the Election Commission of India for:
- **2024 General Elections**
- **2019 General Elections**  
- **2014 General Elections**

### Objectives:
1. Understand the structure of each dataset
2. Identify common constituencies across all 3 years
3. Check data availability (especially gender-wise voter bifurcation)
4. Select 10 constituencies for the dashboard

## 1. Import Required Libraries

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set display options for pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set style for visualizations
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Define File Paths

Defining paths for both Excel and PDF files for all three election years.

In [2]:
# Define file paths
base_path = "/workspaces/-Data-Visualization-Storytelling-assignment/dataset"

# File paths for each year
file_2024 = os.path.join(base_path, "2024", "13-PC-Wise-Voters-Turn-Out.xls")
file_2019 = os.path.join(base_path, "2019", "13. PC Wise Voters Turn Out.xls")
file_2014 = os.path.join(base_path, "2014", "PC wise Voters Turn Out.xlsx")

# Check if files exist
print("Checking file existence:")
print(f"2024 file exists: {os.path.exists(file_2024)}")
print(f"2019 file exists: {os.path.exists(file_2019)}")
print(f"2014 file exists: {os.path.exists(file_2014)}")

Checking file existence:
2024 file exists: True
2019 file exists: True
2014 file exists: True


In [11]:
# ============================================================
# APPROACH: Extract tables directly from PDF files
# This is cleaner than OCR and more reliable than corrupted Excel
# ============================================================

import pdfplumber
import tabula

# Define PDF file paths
pdf_2024 = os.path.join(base_path, "2024", "13-PC-Wise-Voters-Turn-Out.pdf")
pdf_2019 = os.path.join(base_path, "2019", "13. PC Wise Voters Turn Out.pdf")
pdf_2014 = os.path.join(base_path, "2014", "PC wise Voters Turn Out.pdf")

print("=" * 60)
print("PDF FILE CHECK")
print("=" * 60)
print(f"2024 PDF exists: {os.path.exists(pdf_2024)}")
print(f"2019 PDF exists: {os.path.exists(pdf_2019)}")
print(f"2014 PDF exists: {os.path.exists(pdf_2014)}")

PDF FILE CHECK
2024 PDF exists: True
2019 PDF exists: True
2014 PDF exists: True


In [12]:
# ============================================================
# METHOD 1: Try pdfplumber first (usually more accurate for structured tables)
# ============================================================

print("=" * 60)
print("EXTRACTING TABLES USING PDFPLUMBER")
print("=" * 60)

# Let's first examine the structure of one PDF
with pdfplumber.open(pdf_2024) as pdf:
    print(f"\n2024 PDF: {len(pdf.pages)} pages")
    
    # Look at first page to understand structure
    first_page = pdf.pages[0]
    print(f"First page size: {first_page.width} x {first_page.height}")
    
    # Extract tables from first page
    tables = first_page.extract_tables()
    print(f"Tables found on first page: {len(tables)}")
    
    if tables:
        print("\nFirst table preview (first 5 rows):")
        for row in tables[0][:5]:
            print(row)

EXTRACTING TABLES USING PDFPLUMBER

2024 PDF: 20 pages
First page size: 841.89 x 595.28
Tables found on first page: 1

First table preview (first 5 rows):
['SL.\nNO.', 'PC\nNo.', 'PC Name', 'Polling\nStations', 'Electors', None, None, None, None, 'Voters', None, None, None, None, None, None, 'Voter\nTurn\nOut\n(%)', 'Voter Turn Out\n(Excl. Postal) %', None, None]
[None, None, None, None, 'Electors (Including Service Electors)', None, None, None, 'Service\nElectors', 'EVM', None, None, None, None, 'Postal\nVotes', 'Total\nVoters', None, 'Male', 'Female', 'TG']
[None, None, None, None, 'Male', 'Female', 'TG', 'Total', None, 'Male', 'Female', 'TG', 'Total', 'NRI', None, None, None, None, None, None]
['Andhra Pradesh', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['1.', '1', 'Araku (ST)', '2052', '756088', '800954', '111', '1557153', '2520', '560845', '582168', '0', '1143013', '0', '22774', '1165787', '74.87', '74.18', '7

## 3. Extract Data from PDFs

Using pdfplumber to extract tables from PDF files (more reliable than corrupted Excel files).

In [13]:
# ============================================================
# EXTRACT ALL TABLES FROM 2024 PDF
# ============================================================

def extract_all_tables_from_pdf(pdf_path):
    """Extract all tables from all pages of a PDF"""
    all_rows = []
    
    with pdfplumber.open(pdf_path) as pdf:
        print(f"Processing {len(pdf.pages)} pages...")
        
        for page_num, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            
            for table in tables:
                for row in table:
                    # Skip None rows and header rows after first page
                    if row and any(cell is not None for cell in row):
                        all_rows.append(row)
        
    return all_rows

# Extract 2024 data
print("=" * 60)
print("EXTRACTING 2024 DATA")
print("=" * 60)

raw_2024 = extract_all_tables_from_pdf(pdf_2024)
print(f"Total rows extracted: {len(raw_2024)}")

# Show first 10 rows to understand structure
print("\nFirst 10 rows:")
for i, row in enumerate(raw_2024[:10]):
    print(f"{i}: {row}")

EXTRACTING 2024 DATA
Processing 20 pages...
Total rows extracted: 674

First 10 rows:
0: ['SL.\nNO.', 'PC\nNo.', 'PC Name', 'Polling\nStations', 'Electors', None, None, None, None, 'Voters', None, None, None, None, None, None, 'Voter\nTurn\nOut\n(%)', 'Voter Turn Out\n(Excl. Postal) %', None, None]
1: [None, None, None, None, 'Electors (Including Service Electors)', None, None, None, 'Service\nElectors', 'EVM', None, None, None, None, 'Postal\nVotes', 'Total\nVoters', None, 'Male', 'Female', 'TG']
2: [None, None, None, None, 'Male', 'Female', 'TG', 'Total', None, 'Male', 'Female', 'TG', 'Total', 'NRI', None, None, None, None, None, None]
3: ['Andhra Pradesh', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
4: ['1.', '1', 'Araku (ST)', '2052', '756088', '800954', '111', '1557153', '2520', '560845', '582168', '0', '1143013', '0', '22774', '1165787', '74.87', '74.18', '72.68', '0']
5: ['2.', '2', 'Srikakulam', '2049', '8217

In [14]:
# ============================================================
# CLEAN AND PROCESS 2024 DATA
# ============================================================

def clean_2024_data(raw_rows):
    """Clean and structure 2024 data"""
    
    # Define column names based on the header structure we saw
    columns = [
        'SL_NO', 'PC_No', 'PC_Name', 'Polling_Stations',
        'Electors_Male', 'Electors_Female', 'Electors_TG', 'Electors_Total',
        'Service_Electors',
        'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Voters_NRI',
        'Postal_Votes', 'Total_Voters',
        'Turnout_Percent',
        'Turnout_Male', 'Turnout_Female', 'Turnout_TG'
    ]
    
    cleaned_rows = []
    current_state = None
    
    for row in raw_rows:
        # Skip header rows (identified by specific text patterns)
        if row[0] and ('SL.' in str(row[0]) or 'Electors' in str(row[0]) or row[0] is None):
            continue
        if row[1] is None and row[2] is None:  # State name row
            current_state = row[0]
            continue
        
        # Data row - should have PC No in second column
        try:
            if row[1] and str(row[1]).strip().isdigit():
                # This is a valid data row
                data_row = {
                    'State': current_state,
                    'SL_NO': row[0],
                    'PC_No': row[1],
                    'PC_Name': row[2],
                    'Polling_Stations': row[3],
                    'Electors_Male': row[4],
                    'Electors_Female': row[5],
                    'Electors_TG': row[6],
                    'Electors_Total': row[7],
                    'Service_Electors': row[8],
                    'Voters_Male': row[9],
                    'Voters_Female': row[10],
                    'Voters_TG': row[11],
                    'Voters_Total': row[12],
                    'Voters_NRI': row[13],
                    'Postal_Votes': row[14],
                    'Total_Voters': row[15],
                    'Turnout_Percent': row[16],
                    'Turnout_Male': row[17],
                    'Turnout_Female': row[18],
                    'Turnout_TG': row[19]
                }
                cleaned_rows.append(data_row)
        except (IndexError, ValueError) as e:
            continue
    
    return pd.DataFrame(cleaned_rows)

# Clean 2024 data
df_2024 = clean_2024_data(raw_2024)
print(f"2024 DataFrame Shape: {df_2024.shape}")
print(f"\nColumns: {df_2024.columns.tolist()}")
print(f"\nFirst 10 rows:")
df_2024.head(10)

2024 DataFrame Shape: (542, 21)

Columns: ['State', 'SL_NO', 'PC_No', 'PC_Name', 'Polling_Stations', 'Electors_Male', 'Electors_Female', 'Electors_TG', 'Electors_Total', 'Service_Electors', 'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Voters_NRI', 'Postal_Votes', 'Total_Voters', 'Turnout_Percent', 'Turnout_Male', 'Turnout_Female', 'Turnout_TG']

First 10 rows:


,State,SL_NO,PC_No,PC_Name,Polling_Stations,Electors_Male,Electors_Female,Electors_TG,Electors_Total,Service_Electors,Voters_Male,Voters_Female,Voters_TG,Voters_Total,Voters_NRI,Postal_Votes,Total_Voters,Turnout_Percent,Turnout_Male,Turnout_Female,Turnout_TG
0,Andhra Pradesh,1.,1,Araku (ST),2052,756088,800954,111,1557153,2520,560845,582168,0,1143013,0,22774,1165787,74.87,74.18,72.68,0
1,Andhra Pradesh,2.,2,Srikakulam,2049,821758,824812,116,1646686,15512,583439,630675,14,1214128,5,27041,1241169,75.37,71,76.46,12.07
2,Andhra Pradesh,3.,3,Vizianagaram,1886,791124,799780,90,1590994,5788,636780,648518,24,1285322,0,22310,1307632,82.19,80.49,81.09,26.67
3,Andhra Pradesh,4.,4,Visakhapatnam,1962,952279,979004,115,1931398,4095,666349,704170,41,1370560,14,20570,1391130,72.03,69.97,71.93,35.65
4,Andhra Pradesh,5.,5,Anakapalle,1828,784191,817141,33,1601365,4449,638070,671515,16,1309601,0,19125,1328726,82.97,81.37,82.18,48.48
5,Andhra Pradesh,6.,6,Kakinada,1640,805234,829501,186,1634921,799,652890,659778,102,1312770,0,18616,1331386,81.43,81.08,79.54,54.84
6,Andhra Pradesh,7.,7,Amalapuram (SC),1644,759898,772310,21,1532229,819,644627,640319,11,1284957,0,16977,1301934,84.97,84.83,82.91,52.38
7,Andhra Pradesh,8.,8,Rajahmundry,1577,792975,830780,97,1623852,703,647738,666093,60,1313891,9,15271,1329162,81.85,81.68,80.18,61.86
8,Andhra Pradesh,9.,9,Narsapuram,1463,722388,751345,78,1473811,888,602718,614586,47,1217351,28,15202,1232553,83.63,83.43,81.8,60.26
9,Andhra Pradesh,10.,10,Eluru,1744,799901,838089,126,1638116,686,676968,693753,54,1370775,24,17215,1387990,84.73,84.63,82.78,42.86


In [15]:
# ============================================================
# EXTRACT AND EXAMINE 2019 PDF STRUCTURE
# ============================================================

print("=" * 60)
print("EXTRACTING 2019 DATA")
print("=" * 60)

# First, let's see the structure of 2019 PDF
with pdfplumber.open(pdf_2019) as pdf:
    print(f"2019 PDF: {len(pdf.pages)} pages")
    
    # Look at first page
    first_page = pdf.pages[0]
    tables = first_page.extract_tables()
    print(f"Tables found on first page: {len(tables)}")
    
    if tables:
        print("\nFirst table preview (first 8 rows):")
        for i, row in enumerate(tables[0][:8]):
            print(f"{i}: {row}")

EXTRACTING 2019 DATA
2019 PDF: 28 pages
Tables found on first page: 2

First table preview (first 8 rows):
0: ['SL. NO.', 'PC No.', 'PC Name', 'Electors', 'Voters', None, None, None, None, None, None, 'Voter\nTurn Out\n(%)', 'Voter Turn Out (Excl. Postal) %', None, None]
1: [None, None, None, None, 'EVM', None, None, None, None, 'Postal\nVotes', 'Total\nVotes', None, 'Male', 'Female', 'Third Gender']
2: [None, None, None, None, 'Male', 'Female', 'Third Gender', 'Total', 'NRI', None, None, None, None, None, None]
3: ['Andaman & Nicobar Islands', None, None, None, None, None, None, None, None, None, None, None, None, None, None]
4: ['1.', '1', 'Andaman & Nicobar\nIslands', '318471', '109407', '97671', '2', '207080', '0', '318', '207398', '65.12', '64.63', '65.47', '22.22']
5: ['State Total:', '', '', '318471', '109407', '97671', '2', '207080', '0', '318', '207398', '65.12', '64.63', '65.47', '22.22']


## 4. Extract and Clean 2019 Data

Extracting 2019 election data from PDF. Note: 2019 has different structure - no gender-wise electors available.

In [16]:
# ============================================================
# EXTRACT AND CLEAN 2019 DATA
# ============================================================

# Extract all 2019 data
raw_2019 = extract_all_tables_from_pdf(pdf_2019)
print(f"Total rows extracted from 2019: {len(raw_2019)}")

def clean_2019_data(raw_rows):
    """Clean and structure 2019 data"""
    
    cleaned_rows = []
    current_state = None
    
    for row in raw_rows:
        # Skip if row is too short
        if len(row) < 15:
            continue
            
        # Skip header rows
        if row[0] and ('SL.' in str(row[0]) or 'Electors' in str(row[0])):
            continue
        if row[0] is None and row[1] is None:
            continue
            
        # State name row (no PC No in column 1)
        if row[1] is None or row[1] == '' or (row[1] is not None and not str(row[1]).strip().replace('.', '').isdigit()):
            if row[0] and 'Total' not in str(row[0]) and row[2] is None:
                current_state = str(row[0]).strip()
                continue
        
        # Skip total rows
        if row[0] and 'Total' in str(row[0]):
            continue
        
        # Data row
        try:
            if row[1] and str(row[1]).strip().isdigit():
                data_row = {
                    'State': current_state,
                    'SL_NO': row[0],
                    'PC_No': row[1],
                    'PC_Name': str(row[2]).replace('\n', ' ').strip() if row[2] else None,
                    'Electors_Total': row[3],  # Only total electors available
                    'Voters_Male': row[4],
                    'Voters_Female': row[5],
                    'Voters_TG': row[6],
                    'Voters_Total': row[7],
                    'Voters_NRI': row[8],
                    'Postal_Votes': row[9],
                    'Total_Voters': row[10],
                    'Turnout_Percent': row[11],
                    'Turnout_Male': row[12],
                    'Turnout_Female': row[13],
                    'Turnout_TG': row[14]
                }
                cleaned_rows.append(data_row)
        except (IndexError, ValueError) as e:
            continue
    
    return pd.DataFrame(cleaned_rows)

df_2019 = clean_2019_data(raw_2019)
print(f"\n2019 DataFrame Shape: {df_2019.shape}")
print(f"\nColumns: {df_2019.columns.tolist()}")
print(f"\nFirst 10 rows:")
df_2019.head(10)

Processing 28 pages...
Total rows extracted from 2019: 787

2019 DataFrame Shape: (543, 16)

Columns: ['State', 'SL_NO', 'PC_No', 'PC_Name', 'Electors_Total', 'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Voters_NRI', 'Postal_Votes', 'Total_Voters', 'Turnout_Percent', 'Turnout_Male', 'Turnout_Female', 'Turnout_TG']

First 10 rows:


,State,SL_NO,PC_No,PC_Name,Electors_Total,Voters_Male,Voters_Female,Voters_TG,Voters_Total,Voters_NRI,Postal_Votes,Total_Voters,Turnout_Percent,Turnout_Male,Turnout_Female,Turnout_TG
0,Andaman & Nicobar Islands,1.,1,Andaman & Nicobar Islands,318471,109407,97671,2,207080,0,318,207398,65.12,64.63,65.47,22.22
1,Andhra Pradesh,2.,1,Aruku,1451418,523085,544258,19,1067362,0,10873,1078235,74.29,73.71,73.39,20.21
2,Andhra Pradesh,3.,2,Srikakulam,1553860,548060,594937,14,1143011,0,21275,1164286,74.93,70.08,77.11,6.7
3,Andhra Pradesh,4.,3,Vizianagaram,1503980,603059,605843,7,1208909,0,13524,1222433,81.28,79.98,80.8,5.69
4,Andhra Pradesh,5.,4,Visakhapatnam,1829300,608306,624713,7,1233026,0,6895,1239921,67.78,66.26,68.57,6.03
5,Andhra Pradesh,6.,5,Anakapalli,1521363,606111,626764,6,1232881,0,7575,1240456,81.54,80.79,81.29,9.09
6,Andhra Pradesh,7.,6,Kakinada,1563930,616003,611330,66,1227399,0,9415,1236814,79.08,79.38,77.61,29.33
7,Andhra Pradesh,8.,7,Amalapuram,1459556,617210,607337,0,1224547,0,10821,1235368,84.64,84.62,83.18,0
8,Andhra Pradesh,9.,8,Rajahmundry,1534256,617706,626974,8,1244688,0,5800,1250488,81.5,81.92,80.37,6.56
9,Andhra Pradesh,10.,9,Narsapuram,1439691,580912,587960,35,1168907,5,10255,1179162,81.9,81.94,80.48,22.88


## 5. Extract and Clean 2014 Data

Extracting 2014 election data from xlsx file (which works properly unlike the .xls files).

In [22]:
# ============================================================
# CLEAN 2014 DATA FROM XLSX FILE
# ============================================================

# Skip header rows (first 4 rows are headers)
df_2014_raw = pd.read_excel(file_2014, engine='openpyxl', skiprows=4)

# Assign proper column names
df_2014_raw.columns = [
    'State', 'PC_No', 'PC_Name', 'Electors_Total',
    'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Voters_NRI',
    'Postal_Votes', 'Total_Voters', 'Turnout_Percent',
    'Turnout_Male', 'Turnout_Female', 'Turnout_TG'
]

# Remove rows where PC_No is NaN (total rows, empty rows)
df_2014 = df_2014_raw[df_2014_raw['PC_No'].notna()].copy()
df_2014 = df_2014[~df_2014['PC_Name'].isna()].copy()

# Convert PC_No to string to match other datasets
df_2014['PC_No'] = df_2014['PC_No'].astype(str)

print(f"2014 DataFrame Shape: {df_2014.shape}")
print(f"\nColumns: {df_2014.columns.tolist()}")
print(f"\nStates in 2014 data: {df_2014['State'].nunique()}")
print(f"\nFirst 10 rows:")
df_2014.head(10)

2014 DataFrame Shape: (543, 15)

Columns: ['State', 'PC_No', 'PC_Name', 'Electors_Total', 'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Voters_NRI', 'Postal_Votes', 'Total_Voters', 'Turnout_Percent', 'Turnout_Male', 'Turnout_Female', 'Turnout_TG']

States in 2014 data: 35

First 10 rows:


,State,PC_No,PC_Name,Electors_Total,Voters_Male,Voters_Female,Voters_TG,Voters_Total,Voters_NRI,Postal_Votes,Total_Voters,Turnout_Percent,Turnout_Male,Turnout_Female,Turnout_TG
0,Andhra Pradesh,1.0,Adilabad,1386282,518481,525482,2,1043965,0,11628,1055593,76.145618,75.486458,75.217465,1.388889
1,Andhra Pradesh,2.0,Peddapalle,1425355,520741,500992,14,1021747,0,3447,1025194,71.925520,71.787233,71.629329,12.068966
2,Andhra Pradesh,3.0,Karimnagar,1550810,550760,569382,0,1120142,0,7083,1127225,72.686209,70.866284,73.635013,0.000000
3,Andhra Pradesh,4.0,Nizamabad,1496193,467172,559329,4,1026505,0,7527,1034032,69.110870,64.497478,72.494482,4.761905
4,Andhra Pradesh,5.0,Zahirabad,1445354,546010,546798,0,1092808,0,6976,1099784,76.090978,76.071707,75.176737,0.000000
5,Andhra Pradesh,6.0,Medak,1536166,605044,583040,0,1188084,0,5464,1193548,77.696551,78.032637,76.663638,0.000000
6,Andhra Pradesh,7.0,Malkajgiri,3183083,877156,741285,0,1618441,0,6418,1624859,51.046705,50.923041,50.796534,0.000000
7,Andhra Pradesh,8.0,Secundrabad,1893741,546289,457397,3,1003689,0,1074,1004763,53.057044,53.965289,51.934676,1.351351
8,Andhra Pradesh,9.0,Hyderabad,1823217,525811,445415,0,971226,0,544,971770,53.299744,54.762339,51.651961,0.000000
9,Andhra Pradesh,10.0,Chelvella,2185164,695012,617108,0,1312120,0,10192,1322312,60.513170,60.291075,59.808298,0.000000


## 6. Data Comparison and Common Constituencies

Comparing data availability across all three years and identifying common constituencies.

In [23]:
# ============================================================
# DATA SUMMARY COMPARISON
# ============================================================

print("=" * 80)
print("DATA AVAILABILITY COMPARISON ACROSS YEARS")
print("=" * 80)

print("\n📊 DATASET SHAPES:")
print(f"   2024: {df_2024.shape[0]} constituencies, {df_2024.shape[1]} columns")
print(f"   2019: {df_2019.shape[0]} constituencies, {df_2019.shape[1]} columns")
print(f"   2014: {df_2014.shape[0]} constituencies, {df_2014.shape[1]} columns")

print("\n📋 COLUMNS AVAILABLE:")
print("\n   2024 Columns:")
for col in df_2024.columns:
    print(f"      - {col}")

print("\n   2019 Columns:")
for col in df_2019.columns:
    print(f"      - {col}")

print("\n   2014 Columns:")
for col in df_2014.columns:
    print(f"      - {col}")

print("\n" + "=" * 80)
print("⚠️  KEY OBSERVATION: Gender-wise ELECTORS")
print("=" * 80)
print("""
   ✅ 2024: Has Electors_Male, Electors_Female, Electors_TG (AVAILABLE)
   ❌ 2019: Only has Electors_Total (NEEDS CALCULATION)
   ❌ 2014: Only has Electors_Total (NEEDS CALCULATION)
   
   Formula to calculate missing electors:
   Electors_Male = Voters_Male / (Turnout_Male / 100)
   Electors_Female = Voters_Female / (Turnout_Female / 100)
   Electors_TG = Voters_TG / (Turnout_TG / 100)
""")

DATA AVAILABILITY COMPARISON ACROSS YEARS

📊 DATASET SHAPES:
   2024: 542 constituencies, 21 columns
   2019: 543 constituencies, 16 columns
   2014: 543 constituencies, 15 columns

📋 COLUMNS AVAILABLE:

   2024 Columns:
      - State
      - SL_NO
      - PC_No
      - PC_Name
      - Polling_Stations
      - Electors_Male
      - Electors_Female
      - Electors_TG
      - Electors_Total
      - Service_Electors
      - Voters_Male
      - Voters_Female
      - Voters_TG
      - Voters_Total
      - Voters_NRI
      - Postal_Votes
      - Total_Voters
      - Turnout_Percent
      - Turnout_Male
      - Turnout_Female
      - Turnout_TG

   2019 Columns:
      - State
      - SL_NO
      - PC_No
      - PC_Name
      - Electors_Total
      - Voters_Male
      - Voters_Female
      - Voters_TG
      - Voters_Total
      - Voters_NRI
      - Postal_Votes
      - Total_Voters
      - Turnout_Percent
      - Turnout_Male
      - Turnout_Female
      - Turnout_TG

   2014 Columns:
      -

In [24]:
# ============================================================
# FIND COMMON CONSTITUENCIES ACROSS ALL 3 YEARS
# ============================================================

# Clean and standardize PC names for matching
def clean_pc_name(name):
    if pd.isna(name):
        return ""
    return str(name).strip().upper().replace('\n', ' ').replace('  ', ' ')

# Get unique PC names from each year
pc_2024 = set(df_2024['PC_Name'].apply(clean_pc_name))
pc_2019 = set(df_2019['PC_Name'].apply(clean_pc_name))
pc_2014 = set(df_2014['PC_Name'].apply(clean_pc_name))

# Find common constituencies
common_pc = pc_2024 & pc_2019 & pc_2014

print("=" * 80)
print("FINDING COMMON CONSTITUENCIES")
print("=" * 80)
print(f"\n2024 unique constituencies: {len(pc_2024)}")
print(f"2019 unique constituencies: {len(pc_2019)}")
print(f"2014 unique constituencies: {len(pc_2014)}")
print(f"\n✅ COMMON ACROSS ALL 3 YEARS: {len(common_pc)} constituencies")

# Show some examples
print(f"\nSample of common constituencies (first 20):")
for i, pc in enumerate(sorted(common_pc)[:20], 1):
    print(f"   {i}. {pc}")

FINDING COMMON CONSTITUENCIES

2024 unique constituencies: 539
2019 unique constituencies: 540
2014 unique constituencies: 540

✅ COMMON ACROSS ALL 3 YEARS: 382 constituencies

Sample of common constituencies (first 20):
   1. AHMEDABAD EAST
   2. AJMER
   3. AKBARPUR
   4. AKOLA
   5. ALAPPUZHA
   6. ALIGARH
   7. ALLAHABAD
   8. ALWAR
   9. AMBEDKAR NAGAR
   10. AMETHI
   11. AMRELI
   12. AMRITSAR
   13. AMROHA
   14. ANDAMAN & NICOBAR ISLANDS
   15. AONLA
   16. ARAKKONAM
   17. ARANI
   18. ARARIA
   19. ARRAH
   20. ARUNACHAL EAST


In [25]:
# ============================================================
# SELECT 10 CONSTITUENCIES FOR THE DASHBOARD
# ============================================================

# Let's select 10 diverse constituencies from different states
# Criteria: Major cities, different states, common across all years

# First, let's see which major cities/constituencies are available
major_cities = [
    'MUMBAI', 'DELHI', 'BANGALORE', 'CHENNAI', 'KOLKATA', 
    'HYDERABAD', 'AHMEDABAD', 'PUNE', 'LUCKNOW', 'JAIPUR',
    'PATNA', 'VARANASI', 'BHOPAL', 'CHANDIGARH', 'GOA'
]

print("=" * 80)
print("CHECKING AVAILABILITY OF MAJOR CONSTITUENCIES")
print("=" * 80)

available_major = []
for city in major_cities:
    matches = [pc for pc in common_pc if city in pc]
    if matches:
        print(f"\n{city}:")
        for m in matches:
            available_major.append(m)
            print(f"   ✅ {m}")

print(f"\n\nTotal major constituencies available: {len(available_major)}")

CHECKING AVAILABILITY OF MAJOR CONSTITUENCIES

MUMBAI:
   ✅ MUMBAI NORTH WEST
   ✅ MUMBAI NORTH CENTRAL
   ✅ MUMBAI NORTH EAST
   ✅ MUMBAI SOUTH CENTRAL
   ✅ MUMBAI NORTH

DELHI:
   ✅ EAST DELHI
   ✅ SOUTH DELHI
   ✅ WEST DELHI
   ✅ NEW DELHI

BANGALORE:
   ✅ BANGALORE CENTRAL
   ✅ BANGALORE RURAL
   ✅ BANGALORE SOUTH
   ✅ BANGALORE NORTH

CHENNAI:
   ✅ CHENNAI SOUTH
   ✅ CHENNAI CENTRAL
   ✅ CHENNAI NORTH

KOLKATA:
   ✅ KOLKATA DAKSHIN
   ✅ KOLKATA UTTAR

HYDERABAD:
   ✅ HYDERABAD

AHMEDABAD:
   ✅ AHMEDABAD EAST

PUNE:
   ✅ PUNE

LUCKNOW:
   ✅ LUCKNOW

JAIPUR:
   ✅ JAIPUR
   ✅ JAIPUR RURAL

PATNA:
   ✅ VISAKHAPATNAM
   ✅ MACHILIPATNAM
   ✅ PATNA SAHIB

VARANASI:
   ✅ VARANASI

BHOPAL:
   ✅ BHOPAL

CHANDIGARH:
   ✅ CHANDIGARH

GOA:
   ✅ NORTH GOA
   ✅ SOUTH GOA


Total major constituencies available: 32


In [26]:
# ============================================================
# FINAL SELECTION: 10 DIVERSE CONSTITUENCIES
# ============================================================

# Selected 10 constituencies from different regions of India
selected_constituencies = [
    'NEW DELHI',           # North India - Capital
    'MUMBAI NORTH',        # West India - Maharashtra
    'KOLKATA DAKSHIN',     # East India - West Bengal
    'CHENNAI CENTRAL',     # South India - Tamil Nadu
    'BANGALORE SOUTH',     # South India - Karnataka
    'HYDERABAD',           # South India - Telangana
    'LUCKNOW',             # North India - Uttar Pradesh
    'JAIPUR',              # North India - Rajasthan
    'PATNA SAHIB',         # East India - Bihar
    'CHANDIGARH'           # North India - UT
]

print("=" * 80)
print("🎯 SELECTED 10 CONSTITUENCIES FOR DASHBOARD")
print("=" * 80)
for i, pc in enumerate(selected_constituencies, 1):
    print(f"   {i}. {pc}")

print("\n✅ These represent diversity across:")
print("   • Different states/UTs")
print("   • Metro cities and state capitals")
print("   • North, South, East, and West India")

🎯 SELECTED 10 CONSTITUENCIES FOR DASHBOARD
   1. NEW DELHI
   2. MUMBAI NORTH
   3. KOLKATA DAKSHIN
   4. CHENNAI CENTRAL
   5. BANGALORE SOUTH
   6. HYDERABAD
   7. LUCKNOW
   8. JAIPUR
   9. PATNA SAHIB
   10. CHANDIGARH

✅ These represent diversity across:
   • Different states/UTs
   • Metro cities and state capitals
   • North, South, East, and West India


## 7. Save Full Datasets to CSV

Saving the complete cleaned datasets for all constituencies.

In [27]:
# ============================================================
# SAVE CLEANED DATA TO CSV
# ============================================================

# Create output directory
output_dir = "/workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data"
os.makedirs(output_dir, exist_ok=True)

# Save full datasets
df_2024.to_csv(os.path.join(output_dir, "voter_turnout_2024.csv"), index=False)
df_2019.to_csv(os.path.join(output_dir, "voter_turnout_2019.csv"), index=False)
df_2014.to_csv(os.path.join(output_dir, "voter_turnout_2014.csv"), index=False)

print("=" * 80)
print("✅ FULL DATASETS SAVED TO CSV")
print("=" * 80)
print(f"   📁 {output_dir}/voter_turnout_2024.csv ({df_2024.shape[0]} rows)")
print(f"   📁 {output_dir}/voter_turnout_2019.csv ({df_2019.shape[0]} rows)")
print(f"   📁 {output_dir}/voter_turnout_2014.csv ({df_2014.shape[0]} rows)")

✅ FULL DATASETS SAVED TO CSV
   📁 /workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data/voter_turnout_2024.csv (542 rows)
   📁 /workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data/voter_turnout_2019.csv (543 rows)
   📁 /workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data/voter_turnout_2014.csv (543 rows)


In [28]:
# ============================================================
# FILTER AND SAVE SELECTED 10 CONSTITUENCIES
# ============================================================

# Filter each dataset for selected constituencies
def filter_constituencies(df, selected, year):
    """Filter dataframe for selected constituencies"""
    df_copy = df.copy()
    df_copy['PC_Name_Clean'] = df_copy['PC_Name'].apply(clean_pc_name)
    filtered = df_copy[df_copy['PC_Name_Clean'].isin(selected)]
    filtered['Year'] = year
    return filtered.drop('PC_Name_Clean', axis=1)

df_2024_selected = filter_constituencies(df_2024, selected_constituencies, 2024)
df_2019_selected = filter_constituencies(df_2019, selected_constituencies, 2019)
df_2014_selected = filter_constituencies(df_2014, selected_constituencies, 2014)

print("=" * 80)
print("FILTERED DATA FOR SELECTED CONSTITUENCIES")
print("=" * 80)
print(f"   2024: {df_2024_selected.shape[0]} constituencies")
print(f"   2019: {df_2019_selected.shape[0]} constituencies")
print(f"   2014: {df_2014_selected.shape[0]} constituencies")

# Show the selected constituencies from 2024
print("\n2024 Selected Constituencies:")
df_2024_selected[['State', 'PC_Name', 'Turnout_Percent']]

FILTERED DATA FOR SELECTED CONSTITUENCIES
   2024: 10 constituencies
   2019: 10 constituencies
   2014: 10 constituencies

2024 Selected Constituencies:


,State,PC_Name,Turnout_Percent
70,None,Patna Sahib,46.87
147,None,Bangalore South,53.38
224,Maharashtra,Mumbai North,57.2
293,Rajasthan,JAIPUR,64.01
316,Tamil Nadu,CHENNAI CENTRAL,54.19
388,None,Lucknow,52.45
456,West Bengal,Kolkata Dakshin,67.27
514,Telangana,Hyderabad,48.79
524,Chandigarh,Chandigarh,68.03
530,NCT OF Delhi,New Delhi,55.53


In [29]:
# ============================================================
# FIX MISSING STATE NAMES AND CURATE FINAL DATASET
# ============================================================

# Create a mapping for state names based on constituency
state_mapping = {
    'NEW DELHI': 'NCT OF Delhi',
    'MUMBAI NORTH': 'Maharashtra',
    'KOLKATA DAKSHIN': 'West Bengal',
    'CHENNAI CENTRAL': 'Tamil Nadu',
    'BANGALORE SOUTH': 'Karnataka',
    'HYDERABAD': 'Telangana',
    'LUCKNOW': 'Uttar Pradesh',
    'JAIPUR': 'Rajasthan',
    'PATNA SAHIB': 'Bihar',
    'CHANDIGARH': 'Chandigarh'
}

def fix_state_names(df):
    """Fix missing state names based on constituency mapping"""
    df_copy = df.copy()
    for idx, row in df_copy.iterrows():
        if pd.isna(row['State']) or row['State'] is None or row['State'] == 'None':
            pc_clean = clean_pc_name(row['PC_Name'])
            if pc_clean in state_mapping:
                df_copy.at[idx, 'State'] = state_mapping[pc_clean]
    return df_copy

# Fix state names in all datasets
df_2024_selected = fix_state_names(df_2024_selected)
df_2019_selected = fix_state_names(df_2019_selected)
df_2014_selected = fix_state_names(df_2014_selected)

print("State names fixed!")
print("\n2024 Selected Constituencies with States:")
df_2024_selected[['State', 'PC_Name', 'Turnout_Percent', 'Year']]

State names fixed!

2024 Selected Constituencies with States:


,State,PC_Name,Turnout_Percent,Year
70,Bihar,Patna Sahib,46.87,2024
147,Karnataka,Bangalore South,53.38,2024
224,Maharashtra,Mumbai North,57.2,2024
293,Rajasthan,JAIPUR,64.01,2024
316,Tamil Nadu,CHENNAI CENTRAL,54.19,2024
388,Uttar Pradesh,Lucknow,52.45,2024
456,West Bengal,Kolkata Dakshin,67.27,2024
514,Telangana,Hyderabad,48.79,2024
524,Chandigarh,Chandigarh,68.03,2024
530,NCT OF Delhi,New Delhi,55.53,2024


## 8. Data Curation: Calculate Missing Gender-wise Electors

As per assignment instructions, calculating gender-wise electors for 2019 and 2014 using:
- **Electors_Male = Voters_Male / (Turnout_Male / 100)**
- **Electors_Female = Voters_Female / (Turnout_Female / 100)**
- **Electors_TG = Voters_TG / (Turnout_TG / 100)**

In [30]:
# ============================================================
# CALCULATE MISSING GENDER-WISE ELECTORS FOR 2019 AND 2014
# ============================================================

def calculate_electors(df):
    """Calculate gender-wise electors from voters and turnout ratios"""
    df_copy = df.copy()
    
    # Convert to numeric, handling any string values
    for col in ['Voters_Male', 'Voters_Female', 'Voters_TG', 
                'Turnout_Male', 'Turnout_Female', 'Turnout_TG']:
        df_copy[col] = pd.to_numeric(df_copy[col], errors='coerce')
    
    # Calculate electors (avoiding division by zero)
    df_copy['Electors_Male'] = df_copy.apply(
        lambda row: int(row['Voters_Male'] / (row['Turnout_Male'] / 100)) 
        if row['Turnout_Male'] > 0 else 0, axis=1
    )
    
    df_copy['Electors_Female'] = df_copy.apply(
        lambda row: int(row['Voters_Female'] / (row['Turnout_Female'] / 100)) 
        if row['Turnout_Female'] > 0 else 0, axis=1
    )
    
    df_copy['Electors_TG'] = df_copy.apply(
        lambda row: int(row['Voters_TG'] / (row['Turnout_TG'] / 100)) 
        if row['Turnout_TG'] > 0 else 0, axis=1
    )
    
    return df_copy

# Calculate for 2019 and 2014
df_2019_selected = calculate_electors(df_2019_selected)
df_2014_selected = calculate_electors(df_2014_selected)

print("=" * 80)
print("✅ GENDER-WISE ELECTORS CALCULATED FOR 2019 AND 2014")
print("=" * 80)

print("\n2019 Sample (with calculated electors):")
df_2019_selected[['PC_Name', 'Electors_Total', 'Electors_Male', 'Electors_Female', 'Electors_TG', 'Year']].head()

✅ GENDER-WISE ELECTORS CALCULATED FOR 2019 AND 2014

2019 Sample (with calculated electors):


,PC_Name,Electors_Total,Electors_Male,Electors_Female,Electors_TG,Year
71,Patna Sahib,2146008,1133642,1012286,111,2019
82,CHANDIGARH,646729,342199,304502,20,2019
183,Bangalore South,2215489,1153492,1061577,344,2019
261,Mumbai North,1647350,890139,756865,332,2019
293,NEW DELHI,1617470,896288,721158,24,2019


In [31]:
# ============================================================
# CREATE FINAL COMBINED CURATED DATASET
# ============================================================

# Standardize column names and select common columns
common_columns = [
    'Year', 'State', 'PC_Name', 
    'Electors_Total', 'Electors_Male', 'Electors_Female', 'Electors_TG',
    'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total',
    'Postal_Votes', 'Total_Voters',
    'Turnout_Percent', 'Turnout_Male', 'Turnout_Female', 'Turnout_TG'
]

# Ensure 2024 has the right columns
df_2024_selected_clean = df_2024_selected.copy()
# Convert numeric columns
for col in ['Electors_Male', 'Electors_Female', 'Electors_TG', 'Electors_Total',
            'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total',
            'Postal_Votes', 'Total_Voters', 'Turnout_Percent', 
            'Turnout_Male', 'Turnout_Female', 'Turnout_TG']:
    if col in df_2024_selected_clean.columns:
        df_2024_selected_clean[col] = pd.to_numeric(df_2024_selected_clean[col], errors='coerce')

# Select and reorder columns for each year
def prepare_final_df(df, year):
    df_temp = df.copy()
    df_temp['Year'] = year
    
    # Ensure all columns exist
    for col in common_columns:
        if col not in df_temp.columns:
            df_temp[col] = None
    
    return df_temp[common_columns]

df_2024_final = prepare_final_df(df_2024_selected_clean, 2024)
df_2019_final = prepare_final_df(df_2019_selected, 2019)
df_2014_final = prepare_final_df(df_2014_selected, 2014)

# Combine all years
df_combined = pd.concat([df_2024_final, df_2019_final, df_2014_final], ignore_index=True)

# Standardize PC_Name to uppercase
df_combined['PC_Name'] = df_combined['PC_Name'].str.upper().str.strip()

# Sort by constituency and year
df_combined = df_combined.sort_values(['PC_Name', 'Year']).reset_index(drop=True)

print("=" * 80)
print("✅ FINAL CURATED DATASET CREATED")
print("=" * 80)
print(f"Shape: {df_combined.shape}")
print(f"\nColumns: {df_combined.columns.tolist()}")
print(f"\nConstituencies: {df_combined['PC_Name'].nunique()}")
print(f"Years: {sorted(df_combined['Year'].unique())}")

df_combined

✅ FINAL CURATED DATASET CREATED
Shape: (30, 17)

Columns: ['Year', 'State', 'PC_Name', 'Electors_Total', 'Electors_Male', 'Electors_Female', 'Electors_TG', 'Voters_Male', 'Voters_Female', 'Voters_TG', 'Voters_Total', 'Postal_Votes', 'Total_Voters', 'Turnout_Percent', 'Turnout_Male', 'Turnout_Female', 'Turnout_TG']

Constituencies: 10
Years: [np.int64(2014), np.int64(2019), np.int64(2024)]


,Year,State,PC_Name,Electors_Total,Electors_Male,Electors_Female,Electors_TG,Voters_Male,Voters_Female,Voters_TG,Voters_Total,Postal_Votes,Total_Voters,Turnout_Percent,Turnout_Male,Turnout_Female,Turnout_TG
0,2014,Karnataka,BANGALORE SOUTH,1998724,1050583,947604,320,596688,516207,19,1112914,1445,1114359,55.753521,56.795839,54.474971,5.937500
1,2019,Karnataka,BANGALORE SOUTH,2215489,1153492,1061577,344,616311,568793,33,1185137,4520,1189657,53.7,53.430000,53.580000,9.590000
2,2024,Karnataka,BANGALORE SOUTH,2341895,1207404,1134091,400,630756,614381,51,1245188,4864,1250052,53.38,52.240000,54.170000,12.750000
3,2014,Chandigarh,CHANDIGARH,615205,332888,281051,0,244907,208415,0,453322,138,453460,73.708764,73.570390,74.155580,0.000000
4,2019,Chandigarh,CHANDIGARH,646729,342199,304502,20,240635,215557,15,456207,430,456637,70.61,70.320000,70.790000,71.430000
5,2024,Chandigarh,CHANDIGARH,660552,342176,318341,35,234526,213995,27,448548,835,449383,68.03,68.540000,67.220000,77.140000
6,2014,Tamil Nadu,CHENNAI CENTRAL,1328018,665040,662376,252,421903,392161,15,814079,1150,815229,61.386894,63.440149,59.205105,5.952381
7,2019,Tamil Nadu,CHENNAI CENTRAL,1332300,660640,671348,353,399357,383273,56,782686,3071,785757,58.98,60.450000,57.090000,15.820000
8,2024,Tamil Nadu,CHENNAI CENTRAL,1350334,667618,682261,455,370304,357463,135,727902,3808,731710,54.19,55.470000,52.390000,29.670000
9,2014,Andhra Pradesh,HYDERABAD,1823217,960169,862338,0,525811,445415,0,971226,544,971770,53.299744,54.762339,51.651961,0.000000


In [32]:
# ============================================================
# SAVE FINAL CURATED DATASET
# ============================================================

# Save to CSV (for Python work)
csv_path = os.path.join(output_dir, "curated_voter_turnout_10_constituencies.csv")
df_combined.to_csv(csv_path, index=False)

# Save to Excel (for submission)
excel_path = os.path.join(output_dir, "curated_voter_turnout_10_constituencies.xlsx")
df_combined.to_excel(excel_path, index=False, sheet_name='Voter Turnout Data')

print("=" * 80)
print("✅ FINAL CURATED DATASET SAVED")
print("=" * 80)
print(f"\n📁 CSV:   {csv_path}")
print(f"📁 Excel: {excel_path}")
print(f"\n📊 Dataset contains:")
print(f"   • 10 constituencies")
print(f"   • 3 years (2014, 2019, 2024)")
print(f"   • 30 total rows")
print(f"   • 17 variables including gender-wise electors and turnout")

✅ FINAL CURATED DATASET SAVED

📁 CSV:   /workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data/curated_voter_turnout_10_constituencies.csv
📁 Excel: /workspaces/-Data-Visualization-Storytelling-assignment/cleaned_data/curated_voter_turnout_10_constituencies.xlsx

📊 Dataset contains:
   • 10 constituencies
   • 3 years (2014, 2019, 2024)
   • 30 total rows
   • 17 variables including gender-wise electors and turnout


## 9. Variable Types Identification (Assignment Requirement - 1 Mark)

Identifying the type of each variable in the curated dataset as per assignment requirements.

In [33]:
# ============================================================
# VARIABLE TYPES IDENTIFICATION (1 Mark)
# ============================================================

variable_types = {
    'Year': ('Discrete/Ordinal', 'Categorical - Ordered time periods (2014, 2019, 2024)'),
    'State': ('Nominal', 'Categorical - Names of states/UTs'),
    'PC_Name': ('Nominal', 'Categorical - Names of parliamentary constituencies'),
    'Electors_Total': ('Ratio', 'Quantitative - Count of total registered voters'),
    'Electors_Male': ('Ratio', 'Quantitative - Count of male registered voters'),
    'Electors_Female': ('Ratio', 'Quantitative - Count of female registered voters'),
    'Electors_TG': ('Ratio', 'Quantitative - Count of third gender registered voters'),
    'Voters_Male': ('Ratio', 'Quantitative - Count of male votes polled'),
    'Voters_Female': ('Ratio', 'Quantitative - Count of female votes polled'),
    'Voters_TG': ('Ratio', 'Quantitative - Count of third gender votes polled'),
    'Voters_Total': ('Ratio', 'Quantitative - Total EVM votes polled'),
    'Postal_Votes': ('Ratio', 'Quantitative - Count of postal votes'),
    'Total_Voters': ('Ratio', 'Quantitative - Total votes (EVM + Postal)'),
    'Turnout_Percent': ('Ratio', 'Quantitative - Overall voter turnout percentage'),
    'Turnout_Male': ('Ratio', 'Quantitative - Male voter turnout percentage'),
    'Turnout_Female': ('Ratio', 'Quantitative - Female voter turnout percentage'),
    'Turnout_TG': ('Ratio', 'Quantitative - Third gender voter turnout percentage')
}

print("=" * 90)
print("📊 VARIABLE TYPES IDENTIFICATION")
print("=" * 90)
print(f"\n{'Variable':<20} {'Type':<15} {'Description'}")
print("-" * 90)

for var, (var_type, desc) in variable_types.items():
    print(f"{var:<20} {var_type:<15} {desc}")

print("\n" + "=" * 90)
print("📝 SUMMARY:")
print("=" * 90)
print("""
   CATEGORICAL VARIABLES (3):
   • Nominal: State, PC_Name
   • Ordinal: Year
   
   QUANTITATIVE VARIABLES (14):
   • All are RATIO scale (have true zero point)
   • Count variables: Electors, Voters (can be summed, averaged)
   • Percentage variables: Turnout ratios (bounded 0-100%)
""")

📊 VARIABLE TYPES IDENTIFICATION

Variable             Type            Description
------------------------------------------------------------------------------------------
Year                 Discrete/Ordinal Categorical - Ordered time periods (2014, 2019, 2024)
State                Nominal         Categorical - Names of states/UTs
PC_Name              Nominal         Categorical - Names of parliamentary constituencies
Electors_Total       Ratio           Quantitative - Count of total registered voters
Electors_Male        Ratio           Quantitative - Count of male registered voters
Electors_Female      Ratio           Quantitative - Count of female registered voters
Electors_TG          Ratio           Quantitative - Count of third gender registered voters
Voters_Male          Ratio           Quantitative - Count of male votes polled
Voters_Female        Ratio           Quantitative - Count of female votes polled
Voters_TG            Ratio           Quantitative - Count of third g

---

## ✅ Data Exploration Complete - Summary

### Notebook Structure:
| Section | Description |
|---------|-------------|
| 1. Import Libraries | pandas, numpy, matplotlib, seaborn, pdfplumber |
| 2. Define File Paths | Excel and PDF paths for 2024, 2019, 2014 |
| 3. Extract PDF Data | Using pdfplumber for 2024 (542 constituencies) |
| 4. Extract 2019 Data | 543 constituencies from PDF |
| 5. Extract 2014 Data | 543 constituencies from xlsx |
| 6. Data Comparison | Find 382 common constituencies across all years |
| 7. Save Full Datasets | CSV files for all constituencies |
| 8. Data Curation | Calculate missing gender-wise electors using formula |
| 9. Variable Types | Identify 17 variables (3 categorical, 14 quantitative) |

### Key Data Curation Formula Applied:
```
Electors_Male = Voters_Male / (Turnout_Male / 100)
Electors_Female = Voters_Female / (Turnout_Female / 100)  
Electors_TG = Voters_TG / (Turnout_TG / 100)
```

### Selected 10 Constituencies:
1. New Delhi (NCT of Delhi)
2. Mumbai North (Maharashtra)
3. Kolkata Dakshin (West Bengal)
4. Chennai Central (Tamil Nadu)
5. Bangalore South (Karnataka)
6. Hyderabad (Telangana)
7. Lucknow (Uttar Pradesh)
8. Jaipur (Rajasthan)
9. Patna Sahib (Bihar)
10. Chandigarh (UT)

### Files Created:
| File | Purpose |
|------|---------|
| `cleaned_data/curated_voter_turnout_10_constituencies.csv` | Main dataset for Python |
| `cleaned_data/curated_voter_turnout_10_constituencies.xlsx` | For submission |
| `cleaned_data/voter_turnout_2024.csv` | Full 2024 data |
| `cleaned_data/voter_turnout_2019.csv` | Full 2019 data |
| `cleaned_data/voter_turnout_2014.csv` | Full 2014 data |

### Next Steps:
- **Notebook 02**: Create 4 visualizations (Task 2)
- **Notebook 03**: Build interactive dashboard (Task 3)